# Ridge Regression

# Introduction

In linear regression techniques, a linear function is sought that models a series of datapoints as *accurately as possible*. The question remains how to define *accurate*. In the conventional linear regression technique, the sum of the squared errors forms a *loss function* that provides a metric as to what is an accurate. This loss function is conventional, but is not the only loss function that could be used.

In this notebook another type of linear regression, called *ridge regression*, is used. This form of regression uses a similar loss function, but with the addition that small weights are prioritised over larger weights.

In what follows, a dataset will be synthesised and then the technique of ridge regression will be applied to it. 

# Defining the model and the loss function

A linear model of the form:
    
$y = w \cdot x$

will be considered, where $x$ is a 100 dimensional vector and $w$ is a vector whose first 10 entries are 1 and its last 90 entries are zero:

$w = [1,1,1,1,1,1,1,1,1,1,0,0,0,....0,0,0]$

In order to generate data that approximately fits this model, points $(x^{(i)}, y^{(i)})$ can be chosen according to:

$y^{(i)} = w \cdot x^{(i)} + e^{(i)}$

with $e^{(i)} \sim N(\mu=0, \sigma=1)$. The additional $e^{(i)}$ represents Gaussian noise for the points that would otherwise be perfectly governed by the relationship $y = w \cdot x$. 

Ideally in our linear regression, the vector $w$ that models these points will have the first 10 entries close to one and the last 90 entries close to zero. In order to implement this, a technique called ridge reression needs to be implemented. In ridge regression, the loss function can be written as:

$L(w) = \Sigma_{i} (y^{(i)} - (w_1 x_1^{(i)} + ... + w_d x_d^{(i)}))^2 \ + \alpha(w_1^2 + ... + w_d^2 )$

where $\alpha$ is a positive coefficient. The idea behind ridge regression is that weights that are smaller are prioritised because they reduce the loss.

## Creating the Dataset

First import the needed libraries for this notebook.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error


In order to demonstrate the technique of ridge regression, a dataset needs to be created. The function `generate_data` does this. For example, the call `generate_data(number_of_points = 200, number_of_dimensions = 100)` returns 200 points of dimension 100 that are generated by $y^{(i)} = w \cdot x^{(i)} + e^{(i)}$ with $e^{(i)} \sim N(\mu=0, \sigma=1)$ and $w$ a vector whose first 10 entires are 1 and its remaining entries are zero.

In [2]:
def generate_data(number_of_points, number_of_dimensions):
    
    w = np.zeros(number_of_dimensions)

    # Make the first 10 components of w equal to one. The rest will stay zero.
    for i in range(10):
        w[i]=1

    X = np.random.normal(size=(number_of_points, number_of_dimensions)) 
    
    y = np.dot(X, w) + np.random.normal(size = number_of_points)
    
        
    return X, y

A dataset can be generated by a call to this function.

In [3]:
X, y = generate_data(number_of_points=200, number_of_dimensions=100)

A test dataset comprising of 10% of the original data can be witheld. 

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=0)

## Applying Ridge Regression to the Dataset 

In a first effort, the alpha value in the loss function will be kept at a 1.0. The ridge regression model can be constructed by calling `Ridge` from `sklearn.linear_model`.

In [5]:
alpha = 1.0
reg = Ridge(alpha=alpha)

The model can be fit to the training data.

In [6]:
reg.fit(X_train, y_train)

Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

The coefficients, $w$, of the model can be found.

In [7]:
reg.coef_

array([ 0.93175402,  1.03852502,  1.14688153,  1.14671353,  1.01591362,
        0.96286916,  0.9908866 ,  0.74204582,  0.71007161,  1.00097678,
        0.0086987 ,  0.07092882,  0.03005427,  0.05483077, -0.04690283,
        0.05102834, -0.03554497, -0.14976768,  0.20439864, -0.018589  ,
       -0.17657687,  0.06349957, -0.05238249, -0.05075177,  0.00454718,
        0.19794064, -0.0259413 ,  0.01216122, -0.12173588, -0.01895408,
        0.00495658,  0.02855822, -0.13432454, -0.02171785,  0.10204936,
       -0.03197143, -0.20861917, -0.13102983, -0.02473363, -0.06612684,
       -0.153332  , -0.0660188 , -0.00428363, -0.20590617,  0.10473827,
       -0.26963921,  0.15522116,  0.11579502, -0.18243643,  0.13525745,
        0.11902044, -0.11070771,  0.05947429, -0.01591181,  0.01338824,
        0.17499702,  0.15132118, -0.08811159,  0.03037395, -0.01897913,
       -0.00944754, -0.08404712,  0.02905173, -0.03501473,  0.04129698,
       -0.05568717,  0.18383745, -0.02954668, -0.1118225 , -0.04

As seen from the weights above, the first ten are close to one and the others are much smaller in magnitude. This is exactly what would be expected by the ridge regression technique that favours having smaller coefficients. 
Numerically, the mean and standard deviation of the first ten components is: 

In [8]:
reg.coef_[0:10].mean(), reg.coef_[0:10].std()

(0.9686637700858316, 0.1384687403099908)

Whilst the mean and standard deviation of the other components is:

In [9]:
reg.coef_[10:100].mean(), reg.coef_[10:100].std()

(0.003472884305438479, 0.1081023903467146)

The regression model can be used to predict the values y_train values. The *root mean square error* of these predictions can be determined using the `mean_squared_error` function from `sklearn.metrics`.

In [10]:
y_predictions = reg.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_predictions))

print('The RMSE of the model on the training dataset is: {}'.format(rmse))

The RMSE of the model on the training dataset is: 0.5880707073806555


The RMSE on the train dataset is low which is to be expected. This is because the training data was used to build the model, hence the model should be able to predict the training data accurately. 
More importantly, the RMSE can be determined for the test dataset. 

In [11]:
y_predictions = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_predictions))

print('The RMSE of the model on the test dataset is: {}'.format(rmse))

The RMSE of the model on the test dataset is: 1.6166904106461708


This error is larger on the test set, which is a sign that the underlying model is overfitted itself to the training data. 

## Tuning the alpha parameter

In the previous section it was assumed that the alpha constant was one. In this section, a grid search and the technique of cross validation is used to determine the ideal alpha constant.
To begin, the ideal alpha constant's magnitude will be determined. In order to begin the search, a parameter grid is defined. This parameter grid space will use values of $10^{-4}$ to $10^4$ evenly distributed in logspace.

In [12]:
param_grid = {
 'alpha': np.logspace(-4, 4, 9),
}

A grid can be created using `GridSearchCV` from `sklearn.model_selection`. This grid can be fit to the training data and then the best alpha parameter shown. Note: the loss function that is being used is the `neg_mean_squared_error` which is just the negative of the RMSE.

In [13]:
grid = GridSearchCV(Ridge(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1, return_train_score=False)
grid.fit(X_train, y_train)

print("Best parameters: {}, score: {}".format(grid.best_params_, grid.best_score_))

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Best parameters: {'alpha': 10.0}, score: -2.1698966232575434


[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    2.9s finished


It can be seen that the best alpha value has an order of magnitude of around 10. As such,  a new parameter grid can be established that searches alpha values from 0 to 100.

In [14]:
param_grid = {
 'alpha': np.arange(0, 100)    
}

grid = GridSearchCV(Ridge(), param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1, return_train_score=False)
grid.fit(X_train, y_train)

print("Best parameters: {}, score: {}".format(grid.best_params_, grid.best_score_))

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters: {'alpha': 7}, score: -2.1593575141964356


[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    2.4s finished


The ideal alpha value for the training data is found to be 6. As such, a new ridge regression model can be created and trained on all the training data. 

In [15]:
alpha = 6
reg = Ridge(alpha=alpha)
reg.fit(X_train, y_train)

Ridge(alpha=6, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

The regression model can be used to predict the values y_train values.

In [16]:
y_predictions = reg.predict(X_train)

rmse = np.sqrt(mean_squared_error(y_train, y_predictions))

print('The RMSE of the model on the training dataset is: {}'.format(rmse))

The RMSE of the model on the training dataset is: 0.606814219379828


In [17]:
y_predictions = reg.predict(X_test)
rmse = np.sqrt(mean_squared_error(y_test, y_predictions))

print('The RMSE of the model on the test dataset is: {}'.format(rmse))

The RMSE of the model on the test dataset is: 1.5365351160005354


Rather annoyingly, both these errors are slightly larger than when the alpha value was guessed as 1.0. The effort to pick the alpha value using cross-validation did not make any improvement.  

## Conclusion 

In this notebook, a dataset was synthesised and a ridge regression model was applied to it. The model was able to produced weights that were expected of the underlying linear model that the data was sampled from. 
The root mean square errors of this model were determined, both for the training and test data sets. It was seen that the RMSE for the test data set was higher than for the training data set, as would be expected.
Finally, the ideal alpha value of the ridge regression model was found using a gridsearch and cross validation. Unfortunately, this *ideal* alpha value did not improve either the RMSEs for the training nor test datasets.